In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

https://python.langchain.com/docs/how_to/few_shot_examples_chat/

In [14]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
    {"input": "2 🦜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
to_vectorize

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

[Document(metadata={'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}, page_content='What did the cow say to the moon? nothing at all'),
 Document(metadata={'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}, page_content='What did the cow say to the moon? nothing at all'),
 Document(metadata={'input': '2 🦜 4', 'output': '6'}, page_content='2 🦜 4 6'),
 Document(metadata={'input': '2 🦜 4', 'output': '6'}, page_content='2 🦜 4 6')]

In [ ]:
vectorstore.similarity_search("bob")

In [13]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    # k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "who is the president of the usa"})

[{'input': '2 🦜 2', 'output': '4'},
 {'input': '2 🦜 3', 'output': '5'},
 {'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2 🦜 4', 'output': '6'}]

In [34]:
import json

from langchain_chroma import Chroma 
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

with open("examples.json") as f:
    examples = json.load(f)

to_vectorize = [example.values() for example in examples]
# str_examples = [{"input":json.dumps(example["input"]), "output":json.dumps(example["output"])}for example in examples]
# embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=str_examples)
to_vectorize


[dict_values([{'location': 'The scout hut is large brick building with a hard wooden floor.', 'activity': 'Indoor active games.', 'n': 9}, {'location': 'Scout Hut', 'activity': 'Indoor games', 'risks': [{'hazard': 'Walls and items attached to walls (eg radiators)', 'risks': ['Collisions with walls causing injuries to people.'], 'affects': ['Young people'], 'controls': ['Mark a line on the floor about two metres before the walls at each end to show the safe zone. Explain the safe zone to players (for example, no one should run in the safe zone).', 'Consider the height and position of permanent fixings to walls, can they be changed.', 'Add additional safe zone marking around objects permanently fixed to walls where necessary.']}, {'hazard': 'Tables and chairs (and other obstructions)', 'risks': ['Collisions with obstacles.', 'Tripping on tables and chairs them causing injuries to people.'], 'affects': ['Young people', 'Adult volunteers'], 'controls': ['Store chairs and tables safely out 

In [28]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "scout hut"})

[{'input': '{"location": "The scout hut is large brick building with a hard wooden floor.", "activity": "Indoor active games.", "n": 9}',
  'output': '{"location": "Scout Hut", "activity": "Indoor games", "risks": [{"hazard": "Walls and items attached to walls (eg radiators)", "risks": ["Collisions with walls causing injuries to people."], "affects": ["Young people"], "controls": ["Mark a line on the floor about two metres before the walls at each end to show the safe zone. Explain the safe zone to players (for example, no one should run in the safe zone).", "Consider the height and position of permanent fixings to walls, can they be changed.", "Add additional safe zone marking around objects permanently fixed to walls where necessary."]}, {"hazard": "Tables and chairs (and other obstructions)", "risks": ["Collisions with obstacles.", "Tripping on tables and chairs them causing injuries to people."], "affects": ["Young people", "Adult volunteers"], "controls": ["Store chairs and tables

In [4]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?").to_messages())

[HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 4', additional_kwargs={}, response_metadata={}), AIMessage(content='6', additional_kwargs={}, response_metadata={})]


In [11]:
import json
from scout.risk.prompts import user_prompt
from scout.risk.models import ExampleRiskAssessment
import itertools

with open("assessment_examples.json") as f:
    examples = json.load(f)

examples = [ExampleRiskAssessment(**example).to_messages(user_prompt) for example in examples]


examples = list(itertools.chain(*examples))
examples

[HumanMessage(content='\nLocation: The scout hut is large brick building with a hard wooden floor.\n\nActivity: Indoor active games.\n\nIdentify 9 risk(s). Rank them from highest impact to lowest impact.\n', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, name='583c7447-15d7-44c3-828d-c2ae4ddd6eaa', tool_calls=[{'name': 'RiskAssessment', 'args': {'activity': 'Indoor games', 'location': 'Scout Hut', 'risks': [{'hazard': 'Walls and items attached to walls (eg radiators)', 'risks': ['Collisions with walls causing injuries to people.'], 'affects': ['Young people'], 'controls': ['Mark a line on the floor about two metres before the walls at each end to show the safe zone. Explain the safe zone to players (for example, no one should run in the safe zone).', 'Consider the height and position of permanent fixings to walls, can they be changed.', 'Add additional safe zone marking around objects permanently fixed to walls where nece